In [ ]:
import zipfile

def unzip_file(file_path):
  """Giải nén tệp zip.

  Args:
    file_path: Đường dẫn đến tệp zip.
  """
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall()

# Giải nén tệp zip đầu tiên
unzip_file('/content/VLSP2021.zip')

# Giải nén tệp zip thứ hai
unzip_file('/content/Testing.zip')

In [ ]:
import os
import re

def process_text(text):
    """Xử lý nội dung văn bản để phân tách và gán các cột với nhãn B- và I- cho cả cột chính và nhãn lồng."""
    tokens = re.split(r'(\s+|<ENAMEX.*?>|</ENAMEX>)', text)
    tag_stack = []
    result = []
    inside_entity = False  # Cờ cho nhãn chính (cột 2)
    inside_nested_entity = False  # Cờ cho nhãn lồng (cột 3)

    for token in tokens:
        token = token.strip()
        if not token:
            continue

        if token.startswith('<ENAMEX TYPE='):
            # Bắt đầu một thực thể mới
            current_tag = re.search(r'TYPE="([^"]+)"', token).group(1)
            tag_stack.append(current_tag)
            if len(tag_stack) == 1:
                inside_entity = False  # Nhãn chính
            elif len(tag_stack) == 2:
                inside_nested_entity = False  # Nhãn lồng
        elif token == '</ENAMEX>':
            # Kết thúc một thực thể
            if tag_stack:
                tag_stack.pop()
            if len(tag_stack) == 1:
                inside_nested_entity = False
            if len(tag_stack) == 0:
                inside_entity = False
        else:
            # Cột 2: Nhãn chính (B- và I-)
            if tag_stack:
                main_tag = tag_stack[0]
                ner_tag = f"B-{main_tag}" if not inside_entity else f"I-{main_tag}"
                inside_entity = True
            else:
                ner_tag = "O"

            # Cột 3: Nhãn lồng (B- và I- của nhãn thứ 2)
            if len(tag_stack) > 1:
                nested_tag = tag_stack[1]
                nested_ner_tag = f"B-{nested_tag}" if not inside_nested_entity else f"I-{nested_tag}"
                inside_nested_entity = True
            else:
                nested_ner_tag = "0"

            # Thêm kết quả (token, nhãn chính, nhãn lồng)
            result.append([token, ner_tag, nested_ner_tag])

    return result

def process_files_to_single_file(input_dir, output_file):
    """Xử lý tất cả các tệp .muc trong thư mục đầu vào và dồn vào một tệp duy nhất."""
    with open(output_file, 'w', encoding='utf-8') as output:
        for filename in os.listdir(input_dir):
            if filename.endswith('.muc'):
                input_path = os.path.join(input_dir, filename)

                # Đọc nội dung tệp
                with open(input_path, 'r', encoding='utf-8') as file:
                    text = file.read()

                # Xử lý văn bản
                processed_data = process_text(text)

                for row in processed_data:
                    output.write(" ".join(row) + "\n")
                output.write("\n")  # Dòng trống giữa các tệp

# Đường dẫn thư mục và tệp đầu ra
input_directory = "/content/Train-Muc"  # Thay bằng đường dẫn thư mục
output_file = "/content/train.txt"  # Thay bằng đường dẫn tệp kết quả

# Gọi hàm xử lý
process_files_to_single_file(input_directory, output_file)


In [ ]:
# Đường dẫn thư mục và tệp đầu ra
input_directory = "/content/Test"  # Thay bằng đường dẫ
output_file = "/content/test.txt"  # Thay bằng đường dẫn tệp kết quả

# Gọi hàm xử lý
process_files_to_single_file(input_directory, output_file)


In [ ]:
!pip install python-crfsuite


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.5 MB/s eta 0:00:00


In [ ]:
import pycrfsuite
import time

In [ ]:
import pycrfsuite
import nltk
from nltk.tokenize import word_tokenize

# Đọc dữ liệu từ train.txt và chuyển thành định dạng phù hợp
def read_data(file_path):
    sentences = []
    sentence = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Loại bỏ các dòng trống
            if line.strip():
                word, tag, tag_nested = line.strip().split()
                # Loại bỏ tiền tố B- hoặc I- nếu có trong tag
                clean_tag = tag # Giữ phần sau dấu '-'
                sentence.append((word, clean_tag))
            else:
                if sentence:
                    sentences.append(sentence)
                    sentence = []

    # Đảm bảo đưa câu cuối vào
    if sentence:
        sentences.append(sentence)

    return sentences


# Chuyển các từ và nhãn thành đặc trưng cho CRF
def word2features(sentence, i):
    word = sentence[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.isdigit()': word.isdigit(),
        'word.istitle()': word.istitle(),
        'word[-3:]': word[-3:],  # 3 ký tự cuối
        'word[-2:]': word[-2:],  # 2 ký tự cuối
        'word[:3]': word[:3],    # 3 ký tự đầu
        'word[:2]': word[:2],    # 2 ký tự đầu
        'word.length': len(word),  # Độ dài từ
        'word.contains_digit': any(char.isdigit() for char in word),  # Có chứa số không
        'word.contains_special': any(not char.isalnum() for char in word),  # Có ký tự đặc biệt không
        'word.ispunctuation': word in ['.', ',', ';', '!', '?'],  # Có phải dấu câu không
    }

    if i > 0:
        word1 = sentence[i - 1][0]
        features.update({
            'prev.word.lower()': word1.lower(),
            'prev.word.isupper()': word1.isupper(),
            'prev.word.istitle()': word1.istitle(),
            'prev.word[-3:]': word1[-3:],
            'prev.word[:3]': word1[:3],
        })
    else:
        features['BOS'] = True  # Beginning of sentence

    if i < len(sentence) - 1:
        word1 = sentence[i + 1][0]
        features.update({
            'next.word.lower()': word1.lower(),
            'next.word.isupper()': word1.isupper(),
            'next.word.istitle()': word1.istitle(),
            'next.word[-3:]': word1[-3:],
            'next.word[:3]': word1[:3],
        })
    else:
        features['EOS'] = True  # End of sentence

    return features

# Chuyển dữ liệu thành đặc trưng và nhãn
def prepare_data(sentences):
    X = []
    y = []
    for sentence in sentences:
        X.append([word2features(sentence, i) for i in range(len(sentence))])
        y.append([tag for _, tag in sentence])
    return X, y

# Đọc dữ liệu từ file
train_sentences = read_data('/content/train.txt')

# Chuẩn bị dữ liệu đặc trưng
X_train, y_train = prepare_data(train_sentences)

# Huấn luyện mô hình CRF
trainer = pycrfsuite.Trainer()
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Tinh chỉnh các tham số huấn luyện (nếu cần thiết)

# Điều chỉnh tham số huấn luyện
trainer.set_params({
    'max_iterations': 100,  # Tăng số vòng lặp huấn luyện
    'c1': 1.0,  # Hệ số điều chỉnh độ phức tạp (L1)
    'c2': 1e-3,  # Hệ số điều chỉnh độ phức tạp (L2)
    'feature.possible_transitions': True,
})

start_time = time.time()

# Lưu mô hình đã huấn luyện
trainer.train('ner_model.crfsuite')

end_time = time.time()

# Dự đoán với mô hình đã huấn luyện
def predict(model_file, sentence):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_file)

    # Chuyển câu thành đặc trưng
    X_test = [word2features(sentence, i) for i in range(len(sentence))]
    return tagger.tag(X_test)

# Ví dụ dự đoán

test_sentence = ["Moscow", "là", "thủ", "đô", "của", "Nga"]
predicted_tags = predict('ner_model.crfsuite', test_sentence)

# In kết quả dự đoán
for word, tag in zip(test_sentence, predicted_tags):
    print(f'{word}: {tag}')


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 287662
Seconds required: 4.482

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 1181368.076181
Feature norm: 1.000000
Error norm: 1034303.901967
Active features: 179823
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 38.405

***** Iteration #2 *****
Loss: 939669.858317
Feature norm: 2.310429
Error norm: 332363.661644
Active features: 166687
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 18.609

***** Iteration #3 *****
Loss: 857238.093078
Feature norm: 2.043883
Error norm: 267887.525817
Active features: 170583
Line search trials: 1
Line search step: 1.000000
Second

In [ ]:
training_time = end_time - start_time

print(f"Mô hình được huấn luyện xong trong {training_time:.2f} giây.")

Mô hình được huấn luyện xong trong 2086.87 giây.


In [82]:
X_train[0][0:3]

[{'bias': 1.0,
  'word.lower()': 'sau',
  'word.isupper()': False,
  'word.isdigit()': False,
  'word.istitle()': True,
  'word[-3:]': 'Sau',
  'word[-2:]': 'au',
  'word[:3]': 'Sau',
  'word[:2]': 'Sa',
  'word.length': 3,
  'word.contains_digit': False,
  'word.contains_special': False,
  'word.ispunctuation': False,
  'BOS': True,
  'next.word.lower()': "'sống",
  'next.word.isupper()': False,
  'next.word.istitle()': True,
  'next.word[-3:]': 'ống',
  'next.word[:3]': "'Số"},
 {'bias': 1.0,
  'word.lower()': "'sống",
  'word.isupper()': False,
  'word.isdigit()': False,
  'word.istitle()': True,
  'word[-3:]': 'ống',
  'word[-2:]': 'ng',
  'word[:3]': "'Số",
  'word[:2]': "'S",
  'word.length': 5,
  'word.contains_digit': False,
  'word.contains_special': True,
  'word.ispunctuation': False,
  'prev.word.lower()': 'sau',
  'prev.word.isupper()': False,
  'prev.word.istitle()': True,
  'prev.word[-3:]': 'Sau',
  'prev.word[:3]': 'Sau',
  'next.word.lower()': 'chung',
  'next.word.is

In [ ]:
len(X_train)

1050

In [72]:
y_train[0][0:10]

['O',
 'B-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'O',
 'O',
 'O',
 'B-PRODUCT']

In [74]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

# Hàm đọc dữ liệu test từ file
def read_test_data(file_path):
    sentences = []
    sentence = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Loại bỏ các dòng trống
            if line.strip():
                word, tag = line.strip().split()[:2]
                sentence.append((word, tag))
            else:
                if sentence:
                    sentences.append(sentence)
                    sentence = []

    # Đảm bảo đưa câu cuối vào
    if sentence:
        sentences.append(sentence)

    return sentences

# Chuẩn bị dữ liệu test
test_sentences = read_test_data('/content/test.txt')

# Dự đoán trên tập test
def predict_test_data(model_file, test_sentences):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_file)

    X_test = [[word2features(sentence, i) for i in range(len(sentence))] for sentence in test_sentences]
    y_test = [[tag for _, tag in sentence] for sentence in test_sentences]
    y_pred = [tagger.tag(xseq) for xseq in X_test]

    return y_test, y_pred, X_test

# Dự đoán trên tập dữ liệu test
y_test, y_pred, X = predict_test_data('ner_model.crfsuite', test_sentences)

# Chuyển dữ liệu thành dạng phẳng để tính các chỉ số
y_test_flat = [tag for sentence in y_test for tag in sentence]
y_pred_flat = [tag for sentence in y_pred for tag in sentence]

In [76]:
# # Danh sách mới để lưu kết quả
# y_pred_flat_bio = []

# # Biến tạm lưu nhãn trước đó
# previous_label = None

# # Duyệt qua từng nhãn trong danh sách
# for i, label in enumerate(y_pred_flat):
#     if label == 'O':
#         y_pred_flat_bio.append('O')  # Giữ nguyên nếu là nhãn 'O'
#         previous_label = None  # Đặt lại nhãn trước
#     else:
#         # Nếu nhãn hiện tại khác nhãn trước đó hoặc sau 'O', gán B-
#         if label != previous_label:
#             y_pred_flat_bio.append(f"B-{label}")
#         else:
#             # Nếu nhãn giống nhãn trước đó, gán I-
#             y_pred_flat_bio.append(f"I-{label}")
#         previous_label = label  # Cập nhật nhãn trước đó


In [77]:
# y_pred_flat = y_pred_flat_bio

In [78]:
# Sắp xếp lại nhãn để các nhãn B- và I- của cùng một loại xếp cạnh nhau
unique_tags = sorted(set(y_test_flat + y_pred_flat), key=lambda x: (x[2:], x[:2]))  # Sắp xếp theo loại nhãn và B/I

# Báo cáo kết quả
print("Classification Report:")
print(classification_report(y_test_flat, y_pred_flat, labels=unique_tags, digits=4))

# Tính Precision, Recall, và F1
precision, recall, f1, _ = precision_recall_fscore_support(y_test_flat, y_pred_flat, average='weighted')
print(f"Weighted Precision: {precision:.4f}")
print(f"Weighted Recall: {recall:.4f}")
print(f"Weighted F1-Score: {f1:.4f}")

Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

                       precision    recall  f1-score   support

                    O     0.9438    0.9884    0.9656    172690
            B-ADDRESS     0.5455    0.2609    0.3529        23
            I-ADDRESS     0.5702    0.2782    0.3740       248
           B-DATETIME     0.4710    0.5328    0.5000       625
           I-DATETIME     0.5107    0.5385    0.5242       663
      B-DATETIME-DATE     0.6823    0.5508    0.6095       581
      I-DATETIME-DATE     0.6988    0.5506    0.6159       514
 B-DATETIME-DATERANGE     0.3678    0.2254    0.2795       142
 I-DATETIME-DATERANGE     0.6360    0.3576    0.4578       425
  B-DATETIME-DURATION     0.8669    0.5184    0.6488       490
  I-DATETIME-DURATION     0.8201    0.5000    0.6212       556
       B-DATETIME-SET     0.5000    0.5000    0.5000         4
       I-DATETIME-SET     0.6875    0.5500    0.6111        20
      B-DATETIME-TIME     0.6383    0.5556    0.5941        54
      I-DATETIME-TIME     0.5333    0.2254    0.3168  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [79]:
filtered_pairs = [(true, pred) for true, pred in zip(y_test_flat, y_pred_flat) if true != "O"]
filtered_y_test, filtered_y_pred = zip(*filtered_pairs)

print(f"Length of filtered_y_test: {len(filtered_y_test)}")
print(f"Length of filtered_y_pred: {len(filtered_y_pred)}")


Length of filtered_y_test: 38251
Length of filtered_y_pred: 38251


In [80]:
precision, recall, f1, _ = precision_recall_fscore_support(
    filtered_y_test,
    filtered_y_pred,
    labels=unique_tags,
    average='weighted'
)

print(f"Weighted Precision (Excluding 'O'): {precision:.4f}")
print(f"Weighted Recall (Excluding 'O'): {recall:.4f}")
print(f"Weighted F1-Score (Excluding 'O'): {f1:.4f}")


Weighted Precision (Excluding 'O'): 0.7108
Weighted Recall (Excluding 'O'): 0.5033
Weighted F1-Score (Excluding 'O'): 0.5629


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


DƯỚI LÀ CODE ĐỂ TEST THỬ VÀI CÂU:

In [81]:
def extract_features(sentence, i):
    if isinstance(sentence[i], tuple):  # Trường hợp có nhãn (tuple)
        word = sentence[i][0]
    else:  # Trường hợp không có nhãn (chỉ là chuỗi)
        word = sentence[i]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.isdigit()': word.isdigit(),
        'word.istitle()': word.istitle(),
        'word[-3:]': word[-3:],  # 3 ký tự cuối
        'word[-2:]': word[-2:],  # 2 ký tự cuối
        'word[:3]': word[:3],    # 3 ký tự đầu
        'word[:2]': word[:2],    # 2 ký tự đầu
        'word.length': len(word),  # Độ dài từ
        'word.contains_digit': any(char.isdigit() for char in word),  # Có chứa số không
        'word.contains_special': any(not char.isalnum() for char in word),  # Có ký tự đặc biệt không
        'word.ispunctuation': word in ['.', ',', ';', '!', '?'],  # Có phải dấu câu không
    }

    if i > 0:
        if isinstance(sentence[i - 1], tuple):
            prev_word = sentence[i - 1][0]
        else:
            prev_word = sentence[i - 1]
        features.update({
            'prev.word.lower()': prev_word.lower(),
            'prev.word.isupper()': prev_word.isupper(),
            'prev.word.istitle()': prev_word.istitle(),
            'prev.word[-3:]': prev_word[-3:],
            'prev.word[:3]': prev_word[:3],
        })
    else:
        features['BOS'] = True  # Beginning of sentence

    if i < len(sentence) - 1:
        if isinstance(sentence[i + 1], tuple):
            next_word = sentence[i + 1][0]
        else:
            next_word = sentence[i + 1]
        features.update({
            'next.word.lower()': next_word.lower(),
            'next.word.isupper()': next_word.isupper(),
            'next.word.istitle()': next_word.istitle(),
            'next.word[-3:]': next_word[-3:],
            'next.word[:3]': next_word[:3],
        })
    else:
        features['EOS'] = True  # End of sentence

    return features

def preprocess_sentence_without_labels(sentence):
    """
    Chuyển câu đầu vào dạng chuỗi thành định dạng giống test_sentences nhưng không có nhãn.
    """
    # Tách câu thành các từ
    words = sentence.split()
    # Tạo định dạng với mỗi từ là một phần tử trong tuple
    test_sentence = [(word,) for word in words]
    return test_sentence

def predict_unlabeled_data(model_file, sentences):
    """
    Dự đoán nhãn cho các câu không có nhãn.
    Args:
        model_file: Tệp mô hình CRF đã huấn luyện.
        sentences: Danh sách các câu, mỗi câu là danh sách các từ (tokens).
    Returns:
        y_pred: Danh sách các nhãn dự đoán.
        X_test: Các đặc trưng đã tạo ra cho các câu.
    """
    tagger = pycrfsuite.Tagger()
    tagger.open(model_file)

    # Tạo đặc trưng từ dữ liệu đầu vào
    X_test = [[extract_features(sentence, i) for i in range(len(sentence))] for sentence in sentences]
    y_pred = [tagger.tag(xseq) for xseq in X_test]

    return y_pred, X_test

# Dự đoán cho câu không có nhãn
sentences = "Sofascore là hãng thống kê hàng đầu thế giới, do người Croatia sáng lập năm 2010 và có hàng chục triệu người dùng. Theo mô hình của hãng, Xuân Son nhận điểm 10 và dĩ nhiên là cầu thủ hay nhất trận. Vĩ Hào đứng thứ hai với 8,4 điểm, nhờ một bàn và một đường kiến tạo, còn Nguyễn Quang Hải nhận 8,2 điểm với một bàn"
test_sentences = sentences.split()  # Tách câu thành các từ

# Dự đoán nhãn
y_pred, _ = predict_unlabeled_data('ner_model.crfsuite', [test_sentences])

# In kết quả với từ và nhãn dự đoán
for word, label in zip(test_sentences, y_pred[0]):
    print(f"Word: {word}, Label: {label}")


Word: Sofascore, Label: O
Word: là, Label: O
Word: hãng, Label: O
Word: thống, Label: O
Word: kê, Label: O
Word: hàng, Label: O
Word: đầu, Label: O
Word: thế, Label: O
Word: giới,, Label: O
Word: do, Label: O
Word: người, Label: O
Word: Croatia, Label: B-LOCATION-GPE
Word: sáng, Label: O
Word: lập, Label: O
Word: năm, Label: B-DATETIME
Word: 2010, Label: I-DATETIME
Word: và, Label: O
Word: có, Label: O
Word: hàng, Label: O
Word: chục, Label: O
Word: triệu, Label: O
Word: người, Label: O
Word: dùng., Label: O
Word: Theo, Label: O
Word: mô, Label: O
Word: hình, Label: O
Word: của, Label: O
Word: hãng,, Label: O
Word: Xuân, Label: B-PERSON
Word: Son, Label: I-PERSON
Word: nhận, Label: O
Word: điểm, Label: O
Word: 10, Label: B-QUANTITY-NUM
Word: và, Label: O
Word: dĩ, Label: O
Word: nhiên, Label: O
Word: là, Label: O
Word: cầu, Label: O
Word: thủ, Label: O
Word: hay, Label: O
Word: nhất, Label: O
Word: trận., Label: O
Word: Vĩ, Label: O
Word: Hào, Label: O
Word: đứng, Label: O
Word: thứ, L